In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy import stats

def random_walk(mu, sigma, num_steps, plot_flag=False):
    """
    Simulate a 1D random walk for num_steps number of steps.
    Each step is drawn from a normal distribution with mean mu and std sigma.
    
    Parameters:
    -----------
    mu : float
        Mean of step distribution
    sigma : float
        Standard deviation of step distribution
    num_steps : int
        Total number of steps walker takes
    plot_flag : bool, optional
        If True, creates figure of walker trajectory (default: False)
    
    Returns:
    --------
    x_final : float
        Distance from origin after taking num_steps
    x : ndarray
        Array of all positions during the walk
    """
    # Initialize vector of walker locations
    x = np.zeros(num_steps + 1)
    
    # Simulate walk
    for i in range(num_steps):
        step = np.random.normal(mu, sigma)
        x[i + 1] = x[i] + step
    
    x_final = x[num_steps]
    
    if plot_flag:
        t = np.arange(num_steps + 1)
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=t, 
            y=x,
            mode='lines',
            name='Position',
            line=dict(color='blue', width=2)
        ))
        
        fig.update_layout(
            title='1-D Random Walk',
            xaxis_title='# steps',
            yaxis_title='Distance from start [au]',
            template='plotly_white',
            hovermode='x unified'
        )
        
        fig.show()
    
    return x_final, x

In [10]:
# Example 1: Single symmetric random walk (unbiased, mu=0)
print("=== Example 1: Single Symmetric Random Walk ===")
x_final, trajectory = random_walk(mu=0, sigma=1, num_steps=1000, plot_flag=True)
print(f"Final position: {x_final:.3f}")
print(f"Expected final position (theory): 0")
print(f"RMS displacement (theory): {np.sqrt(2 * 1**2 / 2 * 1000):.3f}")
print(f"Actual RMS displacement: {np.std(trajectory):.3f}")

=== Example 1: Single Symmetric Random Walk ===


Final position: 46.431
Expected final position (theory): 0
RMS displacement (theory): 31.623
Actual RMS displacement: 10.519


In [13]:
# Example 2: Multiple random walks to show ensemble behavior
def simulate_ensemble(mu, sigma, num_steps, num_walkers):
    """
    Simulate an ensemble of random walkers.
    
    Parameters:
    -----------
    mu : float
        Mean of step distribution
    sigma : float
        Standard deviation of step distribution
    num_steps : int
        Number of steps each walker takes
    num_walkers : int
        Number of independent walkers
    
    Returns:
    --------
    trajectories : ndarray
        Array of shape (num_walkers, num_steps+1) with all trajectories
    """
    trajectories = np.zeros((num_walkers, num_steps + 1))
    
    for i in range(num_walkers):
        for j in range(num_steps):
            step = np.random.normal(mu, sigma)
            trajectories[i, j + 1] = trajectories[i, j] + step
    
    return trajectories

# Simulate ensemble
print("\n=== Example 2: Ensemble of Random Walkers ===")
num_walkers = 500
num_steps = 500
trajectories = simulate_ensemble(mu=0, sigma=1, num_steps=num_steps, num_walkers=num_walkers)

# Plot first 20 trajectories
fig = go.Figure()
t = np.arange(num_steps + 1)

for i in range(min(20, num_walkers)):
    fig.add_trace(go.Scatter(
        x=t,
        y=trajectories[i, :],
        mode='lines',
        line=dict(width=1),
        opacity=0.6,
        showlegend=False
    ))

# Add mean trajectory
mean_trajectory = np.mean(trajectories, axis=0)
fig.add_trace(go.Scatter(
    x=t,
    y=mean_trajectory,
    mode='lines',
    name='Ensemble Mean',
    line=dict(color='red', width=3)
))

fig.update_layout(
    title=f'Ensemble of {min(20, num_walkers)} Random Walks (out of {num_walkers} total)',
    xaxis_title='# steps',
    yaxis_title='Distance from start [au]',
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

print(f"Mean final position: {np.mean(trajectories[:, -1]):.3f}")
print(f"Expected (theory): 0")


=== Example 2: Ensemble of Random Walkers ===


Mean final position: -0.141
Expected (theory): 0


In [14]:
# Example 3: Verify mean-square displacement grows linearly with time
print("\n=== Example 3: Mean-Square Displacement vs Time ===")

# Calculate mean-square displacement at each time step
msd = np.mean(trajectories**2, axis=0)
t = np.arange(num_steps + 1)

# Theoretical prediction: <x^2> = 2Dt where D = sigma^2 / 2
# For our case with tau=1 step, D = sigma^2 / 2 = 0.5
D = 1**2 / 2  # sigma=1, tau=1
theoretical_msd = 2 * D * t

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=t,
    y=msd,
    mode='markers',
    name='Simulated MSD',
    marker=dict(size=4, color='blue')
))

fig.add_trace(go.Scatter(
    x=t,
    y=theoretical_msd,
    mode='lines',
    name=f'Theory: 2Dt (D={D})',
    line=dict(color='red', width=2, dash='dash')
))

fig.update_layout(
    title='Mean-Square Displacement vs Time',
    xaxis_title='Time (# steps)',
    yaxis_title='<x²>',
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

print(f"Diffusion coefficient D (theory): {D}")
print(f"Diffusion coefficient D (from slope): {msd[-1] / (2 * num_steps):.3f}")


=== Example 3: Mean-Square Displacement vs Time ===


Diffusion coefficient D (theory): 0.5
Diffusion coefficient D (from slope): 0.516


In [32]:
# Example 4: Distribution at different times (verify Gaussian)
print("\n=== Example 4: Spatial Distribution at Different Times ===")

# Run more walkers for better statistics
num_walkers_dist = 10000
num_steps_dist = 100
trajectories_dist = simulate_ensemble(mu=0, sigma=1, num_steps=num_steps_dist, num_walkers=num_walkers_dist)

# Choose times to examine
times = [10, 25, 50, 100]

# Determine global x-axis range based on the final time distribution
final_positions = trajectories_dist[:, max(times)]
x_min = final_positions.min()
x_max = final_positions.max()
# Add some padding
x_range = [x_min * 1.1, x_max * 1.1]

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[f't = {t} steps' for t in times]
)

row_col = [(1,1), (1,2), (2,1), (2,2)]

for idx, (t_step, (row, col)) in enumerate(zip(times, row_col)):
    positions = trajectories_dist[:, t_step]
    
    # Histogram with consistent bins across the global range
    hist, bin_edges = np.histogram(positions, bins=50, density=True, range=x_range)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # Theoretical Gaussian - use the global x range
    D = 1**2 / 2
    variance = 2 * D * t_step
    x_theory = np.linspace(x_range[0], x_range[1], 200)
    gaussian = (1 / np.sqrt(2 * np.pi * variance)) * np.exp(-x_theory**2 / (2 * variance))
    
    # Add histogram
    fig.add_trace(
        go.Bar(
            x=bin_centers,
            y=hist,
            name=f'Simulated (t={t_step})',
            marker=dict(color='lightblue', opacity=0.7),
            showlegend=(idx == 0)
        ),
        row=row, col=col
    )
    
    # Add theoretical curve
    fig.add_trace(
        go.Scatter(
            x=x_theory,
            y=gaussian,
            mode='lines',
            name=f'Theory',
            line=dict(color='red', width=2),
            showlegend=(idx == 0)
        ),
        row=row, col=col
    )
    
    # Set consistent x-axis range for all subplots
    fig.update_xaxes(title_text='Position x', row=row, col=col, range=x_range)
    fig.update_yaxes(title_text='Probability Density', row=row, col=col)

fig.update_layout(
    title_text=f'Position Distributions at Different Times (N={num_walkers_dist} walkers)<br><sub>Note: X-axis range fixed to show spreading over time</sub>',
    template='plotly_white',
    height=700,
    showlegend=True
)

fig.show()

print(f"Number of walkers: {num_walkers_dist}")
print(f"X-axis range: [{x_range[0]:.1f}, {x_range[1]:.1f}]")
print(f"\nStandard deviations at different times:")
for t_step in times:
    positions = trajectories_dist[:, t_step]
    std_sim = np.std(positions)
    std_theory = np.sqrt(2 * D * t_step)
    print(f"  t={t_step:3d}: simulated={std_sim:.3f}, theory={std_theory:.3f}")


=== Example 4: Spatial Distribution at Different Times ===


Number of walkers: 10000
X-axis range: [-43.2, 47.6]

Standard deviations at different times:
  t= 10: simulated=3.143, theory=3.162
  t= 25: simulated=5.078, theory=5.000
  t= 50: simulated=7.110, theory=7.071
  t=100: simulated=10.082, theory=10.000


In [16]:
# Example 5: Biased random walk (p ≠ q, or mu ≠ 0)
print("\n=== Example 5: Biased Random Walk ===")

# Positive drift
x_final_biased, trajectory_biased = random_walk(mu=0.1, sigma=1, num_steps=1000, plot_flag=False)

# Compare with unbiased
x_final_unbiased, trajectory_unbiased = random_walk(mu=0, sigma=1, num_steps=1000, plot_flag=False)

fig = go.Figure()
t = np.arange(1001)

fig.add_trace(go.Scatter(
    x=t,
    y=trajectory_unbiased,
    mode='lines',
    name='Unbiased (μ=0)',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=t,
    y=trajectory_biased,
    mode='lines',
    name='Biased (μ=0.1)',
    line=dict(color='red', width=2)
))

# Add theoretical drift line
drift_line = 0.1 * t
fig.add_trace(go.Scatter(
    x=t,
    y=drift_line,
    mode='lines',
    name='Expected drift',
    line=dict(color='red', width=2, dash='dash')
))

fig.update_layout(
    title='Biased vs Unbiased Random Walk',
    xaxis_title='# steps',
    yaxis_title='Position',
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

print(f"Biased walk final position: {x_final_biased:.3f}")
print(f"Expected final position: {0.1 * 1000:.3f}")
print(f"Unbiased walk final position: {x_final_unbiased:.3f}")


=== Example 5: Biased Random Walk ===


Biased walk final position: 103.052
Expected final position: 100.000
Unbiased walk final position: -26.545


In [17]:
# Example 6: Central Limit Theorem - different step distributions
print("\n=== Example 6: Central Limit Theorem with Different Step Distributions ===")

def random_walk_custom_dist(step_sampler, num_steps, num_walkers):
    """
    Random walk with custom step distribution.
    
    Parameters:
    -----------
    step_sampler : callable
        Function that returns a random step when called
    num_steps : int
        Number of steps
    num_walkers : int
        Number of walkers
    
    Returns:
    --------
    trajectories : ndarray
        All trajectories
    """
    trajectories = np.zeros((num_walkers, num_steps + 1))
    
    for i in range(num_walkers):
        for j in range(num_steps):
            trajectories[i, j + 1] = trajectories[i, j] + step_sampler()
    
    return trajectories

num_walkers_clt = 10000
num_steps_clt = 100

# Different step distributions, all with mean=0, variance=1
step_distributions = {
    'Gaussian': lambda: np.random.normal(0, 1),
    'Uniform': lambda: np.random.uniform(-np.sqrt(3), np.sqrt(3)),  # variance = 1
    'Laplace': lambda: np.random.laplace(0, 1/np.sqrt(2)),  # variance = 1
}

fig = go.Figure()

for name, sampler in step_distributions.items():
    trajectories = random_walk_custom_dist(sampler, num_steps_clt, num_walkers_clt)
    final_positions = trajectories[:, -1]
    
    # Histogram
    hist, bin_edges = np.histogram(final_positions, bins=60, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    fig.add_trace(go.Scatter(
        x=bin_centers,
        y=hist,
        mode='lines+markers',
        name=f'{name} steps',
        line=dict(width=2),
        marker=dict(size=4)
    ))

# Theoretical Gaussian (all should converge to this)
D = 1 / 2  # variance = 1, tau = 1
variance = 2 * D * num_steps_clt
x_theory = np.linspace(-30, 30, 200)
gaussian = (1 / np.sqrt(2 * np.pi * variance)) * np.exp(-x_theory**2 / (2 * variance))

fig.add_trace(go.Scatter(
    x=x_theory,
    y=gaussian,
    mode='lines',
    name='Theory (Gaussian)',
    line=dict(color='black', width=3, dash='dash')
))

fig.update_layout(
    title=f'Central Limit Theorem: Different Step Distributions → Same Gaussian (t={num_steps_clt})',
    xaxis_title='Final Position',
    yaxis_title='Probability Density',
    template='plotly_white'
)

fig.show()

print("All distributions converge to Gaussian regardless of step distribution!")


=== Example 6: Central Limit Theorem with Different Step Distributions ===


All distributions converge to Gaussian regardless of step distribution!


In [27]:
# Example 7: 2D Random Walk
print("\n=== Example 7: 2D Random Walk ===")

def random_walk_2d(num_steps, num_walkers=1):
    """
    Simulate 2D random walk(s).
    
    Parameters:
    -----------
    num_steps : int
        Number of steps
    num_walkers : int
        Number of walkers
    
    Returns:
    --------
    x, y : ndarray
        x and y coordinates of trajectories
    """
    x = np.zeros((num_walkers, num_steps + 1))
    y = np.zeros((num_walkers, num_steps + 1))
    
    for i in range(num_walkers):
        for j in range(num_steps):
            x[i, j + 1] = x[i, j] + np.random.normal(0, 1)
            y[i, j + 1] = y[i, j] + np.random.normal(0, 1)
    
    return x, y

# Single 2D walker
x_2d, y_2d = random_walk_2d(num_steps=1000, num_walkers=1)

fig = go.Figure()

# Plot trajectory
fig.add_trace(go.Scatter(
    x=x_2d[0, :],
    y=y_2d[0, :],
    mode='lines',
    name='Path',
    line=dict(color='blue', width=1),
    opacity=0.6
))

# Mark start and end
fig.add_trace(go.Scatter(
    x=[x_2d[0, 0]],
    y=[y_2d[0, 0]],
    mode='markers',
    name='Start',
    marker=dict(size=15, color='green', symbol='star')
))

fig.add_trace(go.Scatter(
    x=[x_2d[0, -1]],
    y=[y_2d[0, -1]],
    mode='markers',
    name='End',
    marker=dict(size=15, color='red', symbol='x')
))

fig.update_layout(
    title='2D Random Walk',
    xaxis_title='x position',
    yaxis_title='y position',
    template='plotly_white',
    yaxis=dict(scaleanchor="x", scaleratio=1),  # Equal aspect ratio
    hovermode='closest'
)

fig.show()

# Calculate distance from origin
r_2d = np.sqrt(x_2d[0, :]**2 + y_2d[0, :]**2)
print(f"Final distance from origin: {r_2d[-1]:.3f}")
print(f"Expected RMS distance (theory, 2D): {np.sqrt(4 * 0.5 * 1000):.3f}")


=== Example 7: 2D Random Walk ===


Final distance from origin: 40.504
Expected RMS distance (theory, 2D): 44.721


In [28]:
# Example 8: Multiple 2D walkers - verify r² = 4Dt
print("\n=== Example 8: 2D Random Walk Ensemble ===")

num_walkers_2d = 1000
num_steps_2d = 500
x_2d_ensemble, y_2d_ensemble = random_walk_2d(num_steps=num_steps_2d, num_walkers=num_walkers_2d)

# Calculate mean-square distance from origin
r_squared = x_2d_ensemble**2 + y_2d_ensemble**2
msd_2d = np.mean(r_squared, axis=0)

t = np.arange(num_steps_2d + 1)
D = 0.5  # sigma=1, tau=1
theoretical_msd_2d = 4 * D * t

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=t,
    y=msd_2d,
    mode='markers',
    name='Simulated <r²>',
    marker=dict(size=4, color='blue')
))

fig.add_trace(go.Scatter(
    x=t,
    y=theoretical_msd_2d,
    mode='lines',
    name=f'Theory: 4Dt (D={D})',
    line=dict(color='red', width=2, dash='dash')
))

fig.update_layout(
    title='2D Random Walk: Mean-Square Distance vs Time',
    xaxis_title='Time (# steps)',
    yaxis_title='<r²> = <x²> + <y²>',
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

print(f"Diffusion coefficient D (theory): {D}")
print(f"Diffusion coefficient D (from 2D slope): {msd_2d[-1] / (4 * num_steps_2d):.3f}")


=== Example 8: 2D Random Walk Ensemble ===


Diffusion coefficient D (theory): 0.5
Diffusion coefficient D (from 2D slope): 0.494


In [29]:
# Example 9: Visualize spreading in 2D over time
print("\n=== Example 9: 2D Spatial Distribution Over Time ===")

# Snapshot positions at different times
times_2d = [10, 50, 100, 250, 500]

fig = go.Figure()

colors = px.colors.sequential.Viridis
n_colors = len(times_2d)

for idx, t_snap in enumerate(times_2d):
    x_positions = x_2d_ensemble[:, t_snap]
    y_positions = y_2d_ensemble[:, t_snap]
    
    fig.add_trace(go.Scatter(
        x=x_positions,
        y=y_positions,
        mode='markers',
        name=f't = {t_snap}',
        marker=dict(
            size=4,
            color=colors[int(idx * len(colors) / n_colors)],
            opacity=0.5
        )
    ))
    
    # Add circle showing theoretical RMS radius
    theta = np.linspace(0, 2*np.pi, 100)
    r_rms = np.sqrt(4 * D * t_snap)
    x_circle = r_rms * np.cos(theta)
    y_circle = r_rms * np.sin(theta)
    
    fig.add_trace(go.Scatter(
        x=x_circle,
        y=y_circle,
        mode='lines',
        name=f't={t_snap} (theory)',
        line=dict(
            color=colors[int(idx * len(colors) / n_colors)],
            width=2,
            dash='dash'
        ),
        showlegend=False
    ))

fig.update_layout(
    title='2D Random Walk: Spatial Distribution at Different Times',
    xaxis_title='x position',
    yaxis_title='y position',
    template='plotly_white',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    hovermode='closest',
    width=800,
    height=800
)

fig.show()

print("\nRMS radii at different times:")
for t_snap in times_2d:
    r = np.sqrt(x_2d_ensemble[:, t_snap]**2 + y_2d_ensemble[:, t_snap]**2)
    r_rms_sim = np.sqrt(np.mean(r**2))
    r_rms_theory = np.sqrt(4 * D * t_snap)
    print(f"  t={t_snap:3d}: simulated={r_rms_sim:.3f}, theory={r_rms_theory:.3f}")


=== Example 9: 2D Spatial Distribution Over Time ===



RMS radii at different times:
  t= 10: simulated=4.427, theory=4.472
  t= 50: simulated=10.085, theory=10.000
  t=100: simulated=14.182, theory=14.142
  t=250: simulated=22.290, theory=22.361
  t=500: simulated=31.426, theory=31.623


In [30]:
# Example 10: Diffusion time scaling (τ_d ∝ ℓ²)
print("\n=== Example 10: Diffusion Time Scaling ===")

# Simulate how long it takes to reach various distances
def time_to_reach_distance(target_distance, D=0.5, num_trials=1000):
    """
    Estimate average time for a random walker to first reach a target distance.
    
    Parameters:
    -----------
    target_distance : float
        Target distance from origin
    D : float
        Diffusion coefficient
    num_trials : int
        Number of trials to average
    
    Returns:
    --------
    mean_time : float
        Average first-passage time
    """
    times = []
    
    for _ in range(num_trials):
        x = 0
        t = 0
        max_steps = 100000  # Safety limit
        
        while abs(x) < target_distance and t < max_steps:
            x += np.random.normal(0, 1)
            t += 1
        
        if t < max_steps:
            times.append(t)
    
    return np.mean(times)

# Test different distances
distances = np.array([1, 2, 3, 5, 7, 10, 15, 20])
times_to_reach = []

print("Computing first-passage times (this may take a moment)...")
for dist in distances:
    t_mean = time_to_reach_distance(dist, D=0.5, num_trials=500)
    times_to_reach.append(t_mean)
    print(f"  Distance {dist:2.0f}: mean time = {t_mean:.1f} steps")

times_to_reach = np.array(times_to_reach)

# Fit to τ ∝ ℓ²
from numpy.polynomial import Polynomial
p = Polynomial.fit(distances**2, times_to_reach, 1)
slope = p.coef[1]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=distances**2,
    y=times_to_reach,
    mode='markers',
    name='Simulated',
    marker=dict(size=10, color='blue')
))

fig.add_trace(go.Scatter(
    x=distances**2,
    y=p(distances**2),
    mode='lines',
    name=f'Fit: τ = {slope:.3f}ℓ²',
    line=dict(color='red', width=2, dash='dash')
))

# Theoretical line: τ = ℓ²/(2D)
theoretical_times = distances**2 / (2 * 0.5)
fig.add_trace(go.Scatter(
    x=distances**2,
    y=theoretical_times,
    mode='lines',
    name='Theory: τ = ℓ²/(2D)',
    line=dict(color='green', width=2, dash='dot')
))

fig.update_layout(
    title='Diffusion Time Scaling: τ ∝ ℓ²',
    xaxis_title='Distance² (ℓ²)',
    yaxis_title='Mean First-Passage Time (τ)',
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

print(f"\nFitted slope: {slope:.3f}")
print(f"Theoretical slope (1/2D): {1/(2*0.5):.3f}")


=== Example 10: Diffusion Time Scaling ===
Computing first-passage times (this may take a moment)...
  Distance  1: mean time = 2.9 steps
  Distance  2: mean time = 6.8 steps
  Distance  3: mean time = 12.5 steps
  Distance  5: mean time = 31.9 steps
  Distance  7: mean time = 59.3 steps
  Distance 10: mean time = 113.4 steps
  Distance 15: mean time = 244.8 steps
  Distance 20: mean time = 382.1 steps



Fitted slope: 192.724
Theoretical slope (1/2D): 1.000


In [31]:
# Example 11: Interactive animation of random walk
print("\n=== Example 11: Animated Random Walk ===")

# Generate a random walk
np.random.seed(42)
num_steps_anim = 200
x_anim = np.zeros(num_steps_anim + 1)
y_anim = np.zeros(num_steps_anim + 1)

for i in range(num_steps_anim):
    x_anim[i + 1] = x_anim[i] + np.random.normal(0, 1)
    y_anim[i + 1] = y_anim[i] + np.random.normal(0, 1)

# Create frames for animation
frames = []
for i in range(0, num_steps_anim + 1, 5):  # Every 5 steps for smoother animation
    frame = go.Frame(
        data=[
            go.Scatter(
                x=x_anim[:i+1],
                y=y_anim[:i+1],
                mode='lines',
                line=dict(color='blue', width=2),
                name='Path'
            ),
            go.Scatter(
                x=[x_anim[i]],
                y=[y_anim[i]],
                mode='markers',
                marker=dict(size=12, color='red'),
                name='Current Position'
            )
        ],
        name=str(i)
    )
    frames.append(frame)

# Initial frame
fig = go.Figure(
    data=[
        go.Scatter(
            x=[x_anim[0]],
            y=[y_anim[0]],
            mode='lines',
            line=dict(color='blue', width=2),
            name='Path'
        ),
        go.Scatter(
            x=[x_anim[0]],
            y=[y_anim[0]],
            mode='markers',
            marker=dict(size=12, color='red'),
            name='Current Position'
        )
    ],
    frames=frames
)

# Add play and pause buttons
fig.update_layout(
    title='Animated 2D Random Walk',
    xaxis_title='x position',
    yaxis_title='y position',
    template='plotly_white',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    updatemenus=[
        dict(
            type='buttons',
            showactive=False,
            buttons=[
                dict(
                    label='Play',
                    method='animate',
                    args=[None, dict(frame=dict(duration=50, redraw=True), fromcurrent=True)]
                ),
                dict(
                    label='Pause',
                    method='animate',
                    args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')]
                )
            ],
            x=0.1,
            y=1.15
        )
    ],
    sliders=[dict(
        steps=[dict(method='animate',
                    args=[[f.name], dict(mode='immediate', frame=dict(duration=0, redraw=True))],
                    label=f.name) for f in frames],
        active=0,
        y=-0.1,
        x=0.1,
        currentvalue=dict(prefix='Step: ', visible=True, xanchor='center'),
        len=0.9
    )]
)

fig.show()

print("Click 'Play' to watch the random walk unfold!")


=== Example 11: Animated Random Walk ===


Click 'Play' to watch the random walk unfold!
